In [7]:
import numpy as np
from time import sleep
from ipycanvas import Canvas, hold_canvas
from dataclasses import dataclass

In [8]:
class Box:
    NORMALS = np.array([[1, 0], [0, 1], [-1, 0], [0, -1]])
    
    def __init__(self, half_extents, density):
        self.mass = half_extents[0] * half_extents[1] * 4 * density
        self.inertia = half_extents[0] * half_extents[1] * (half_extents[0]**2 + half_extents[1]**2) * 4 / 3 * density
        self.half_extents = np.array(half_extents)
        self.corners = np.array([[1, 1], [-1, 1], [-1, -1], [1, -1]]) * self.half_extents
        self.pose = np.zeros(3)
        self.velocity = np.zeros(3)
        self.color = (0, 0, 0)
        self.index = -1
        self.mu = 0

    def get_rotation_matrix(self):
        c = np.cos(self.pose[2])
        s = np.sin(self.pose[2])
        return np.array([[c, -s], [s, c]])
        
    def get_polygon(self):
        return self.corners @ self.get_rotation_matrix().T + self.pose[:2]
    
    def dist(self, points):
        points = np.asanyarray(points)
        R = self.get_rotation_matrix()

        points = (points - self.pose[:2]) @ R  # inverse transform

        dxp = points[:, 0] - self.half_extents[0]
        dxn = -self.half_extents[0] - points[:, 0]
        dyp = points[:, 1] - self.half_extents[1]
        dyn = -self.half_extents[1] - points[:, 1]
        
        xp = points[:, 0] < self.half_extents[0]
        xn = points[:, 0] > -self.half_extents[0]
        yp = points[:, 1] < self.half_extents[1]
        yn = points[:, 1] > -self.half_extents[1]
        
        # classify
        t0 = xp * xn * yp * yn
        t1 = ~xp * yp * yn
        t2 = ~yp * xp * xn
        t3 = ~xn * yp * yn
        t4 = ~yn * xp * xn
        t5 = ~xp * ~yp
        t6 = ~xn * ~yp
        t7 = ~xn * ~yn
        t8 = ~xp * ~yn
        
        dxp = points[:, 0] - self.half_extents[0]
        dxn = -self.half_extents[0] - points[:, 0]
        dyp = points[:, 1] - self.half_extents[1]
        dyn = -self.half_extents[1] - points[:, 1]
        ds = np.vstack((dxp, dyp, dxn, dyn))
        m = ds.argmax(0)

        edge_dist = ds.max(0)
        edge_normal = Box.NORMALS[m]
        
        pd5 = (dxp**2 + dyp**2) ** 0.5
        pd6 = (dxn**2 + dyp**2) ** 0.5
        pd7 = (dxn**2 + dyn**2) ** 0.5
        pd8 = (dxp**2 + dyn**2) ** 0.5

        pn5 = (points - self.half_extents) / pd5[:, None]
        pn6 = (points - self.half_extents * [-1, 1]) / pd6[:, None]
        pn7 = (points - self.half_extents * [-1, -1]) / pd7[:, None]
        pn8 = (points - self.half_extents * [1, -1]) / pd8[:, None]

        te = t0 + t1 + t2 + t3 + t4
        dist = te * edge_dist + t5 * pd5 + t6 * pd6 + t7 * pd7 + t8 * pd8
        normal = te[:, None] * edge_normal + t5[:, None] * pn5 + t6[:, None] * pn6 + t7[:, None] * pn7 + t8[:, None] * pn8
        
        return dist, normal

In [9]:
@dataclass
class Contact:
    body0: Box
    body1: Box
    point: np.ndarray
    normal: np.ndarray
    separation: float
    impulse: np.ndarray = 0

In [10]:
def generate_contact(box1, box2, contact_dist):
    contacts = []
    
    points = box2.get_polygon()
    dist12, normal12 = box1.dist(points)
    for i, d in enumerate(dist12):
        if d < contact_dist:
            contacts.append((points[i], dist12[i], normal12[i]))

    points = box1.get_polygon()
    dist21, normal21 = box2.dist(points)
    for i, d in enumerate(dist21):
        if d < contact_dist:
            contacts.append((points[i], dist21[i], -normal21[i]))
            
    return contacts

In [ ]:
def compute_v_at(point, center, v, w):
    r = point - center
    return v + np.array([-r[1], r[0]]) * w

def cross2d(u, v):
    return np.dot([-u[1], u[0]], v)

def m_eff_inv(m, I, r, n):
    return 1/m + 1/I*cross2d(r, n)**2

class Simulation:
    def __init__(self, width=900, height=500):
        self.width = width
        self.height = height
        self.boxes = []
        self.gravity = np.array([0, -98])

        self.x = np.zeros((0, 3))
        self.v = np.zeros((0, 3))
        self.canvas = Canvas(width=self.width, height=self.height)
        self.contact_distance = 1
        self.solver_iterations = 5
        
    def add_box(self, box):
        self.boxes.append(box)
        
    def step(self, dt):
        if len(self.boxes) != len(self.x):
            self.x = np.zeros((len(self.boxes), 3))
            self.v = np.zeros((len(self.boxes), 3))
        for i, b in enumerate(self.boxes):
            b.index = i
            self.x[i] = b.pose
            self.v[i] = b.velocity
        
        self.generate_contacts()
        self.solve_contacts()
        
        self.preintegration(dt)
        self.integration(dt)

        for i, b in enumerate(self.boxes):
            b.pose = self.x[i]
            b.velocity = self.v[i]
            
    def generate_contacts(self):
        # first, generate contacts with the walls
        
        # floor
        contacts = []
        for i, b in enumerate(self.boxes):
            for p in b.get_polygon():
                if p[1] < self.contact_distance:
                    contacts.append(Contact(None, b, p, np.array([0, 1]), p[1]))
                    
        
        for b1 in range(len(self.boxes)):
            for b2 in range(b1+1, len(self.boxes)):
                box1 = self.boxes[b1]
                box2 = self.boxes[b2]
        
                # box-box
                points = box2.get_polygon()
                dist12, normal12 = box1.dist(points)
                for i, d in enumerate(dist12):
                    if d < self.contact_distance:
                        contacts.append(Contact(box1, box2, points[i], normal12[i], dist12[i]))

                points = box1.get_polygon()
                dist21, normal21 = box2.dist(points)
                for i, d in enumerate(dist21):
                    if d < self.contact_distance:
                        contacts.append(Contact(box1, box2, points[i], -normal21[i], dist21[i]))
                    
        self.contacts = contacts
        
    def solve_contacts(self):
        for it in range(self.solver_iterations):
            for c in self.contacts:

                # solve for normal contact
                if c.body0 is None:
                    vp0 = np.zeros(2)
                    m_inv_0 = 0
                else:
                    r0 = c.point - self.x[c.body0.index, :2]
                    v0 = self.v[c.body0.index]
                    vp0 = v0[:2] + cross2d(r0, v0[2])
                    m_inv_0 = m_eff_inv(c.body0.mass, c.body0.inertia, r0, c.normal)

                r1 = c.point - self.x[c.body1.index, :2]
                v1 = self.v[c.body1.index]
                vp1 = v1[:2] + cross2d(r1, v1[2])
                m_inv_1 = m_eff_inv(c.body1.mass, c.body1.inertia, r1, c.normal)

                rel_v = vp1-vp0
                rel_v_n = c.normal.dot(rel_v)
                if rel_v_n > 0:
                    continue

                impulse = -rel_v_n/(m_inv_1-m_inv_0)
                if c.body0 is not None:
                    self.v[c.body0.index][:2] -= impulse * c.normal / c.body0.mass
                    self.v[c.body0.index][2] -= impulse * cross2d(r1, c.normal) / c.body0.inertia

                self.v[c.body1.index][:2] += impulse * c.normal / c.body1.mass
                self.v[c.body1.index][2] += impulse * cross2d(r1, c.normal) / c.body1.inertia

                # solve for friction
                if c.body0 is None:
                    vp0 = np.zeros(2)
                    mu0 = 0
                else:
                    r0 = c.point - self.x[c.body0.index, :2]
                    v0 = self.v[c.body0.index]
                    vp0 = v0[:2] + cross2d(r0, v0[2])
                    mu0 = c.body0.mu

                r1 = c.point - self.x[c.body1.index, :2]
                v1 = self.v[c.body1.index]
                vp1 = v1[:2] + cross2d(r1, v1[2])
                mu1 = c.body1.mu
                
                mu = (mu0 + mu1) / 2
                max_f_impulse = impulse * mu
                
                rel_v = vp1-vp0
                rel_v_n = c.normal.dot(rel_v)
                rel_v_t = rel_v - rel_v_n * c.normal
                fdir = rel_v_t / (np.linalg.norm(rel_v_t) + 1e-6)

                if c.body0 is None:
                    m_inv_0 = 0
                else:
                    m_inv_0 = m_eff_inv(c.body0.mass, c.body0.inertia, r0, fdir)
                m_inv_1 = m_eff_inv(c.body1.mass, c.body1.inertia, r1, fdir)

                impulse = min(np.linalg.norm(rel_v_t) / (m_inv_1-m_inv_0), max_f_impulse)
                impulse = -fdir * impulse
                if c.body0 is not None:
                    self.v[c.body0.index][:2] -= -impulse / c.body0.mass
                    self.v[c.body0.index][2] -= -cross2d(r1, impulse) / c.body0.inertia

                self.v[c.body1.index][:2] += impulse / c.body1.mass
                self.v[c.body1.index][2] += cross2d(r1, impulse) / c.body1.inertia


    def preintegration(self, dt):
        self.v_ = self.v + np.pad(self.gravity, [0, 1]) * dt
    
    def integration(self, dt):
        self.v = self.v_
        self.x = self.x + self.v * dt
        
    def display(self):
        display(self.canvas)
                
    def clear(self):
        self.canvas.clear()
        
    def draw(self):
        polygons = []
        colors = []
        for box in self.boxes:
            polygons.append(box.get_polygon() * [1, -1] + [0, self.height])
            colors.append(box.color)
        self.canvas.fill_styled_polygons(polygons, color=colors)
        for contact in self.contacts:
            self.canvas.fill_style = '#ff0000'
            self.canvas.fill_circle(contact.point[0], self.height - contact.point[1], 5)

In [30]:
box1 = Box([200, 100], 1)
box1.pose[:2] = [400, 100]
box1.color = (0, 255, 0)
box1.mu = 1.0

box2 = Box([50, 50], 1)
box2.pose[:2] = [400, 300]
box2.color = (0, 0, 255)
box2.velocity[2] = 1
box2.mu = 1.0

sim = Simulation(900, 500)
sim.solver_iterations = 3
sim.add_box(box1)
sim.add_box(box2)

In [31]:
sim.display()
for i in range(1000):
    sim.step(0.01)
    with hold_canvas():
        sim.clear()
        sim.draw()
    sleep(0.01)

Canvas(width=900)